In [50]:
import kagglehub
import os
from pyspark.sql import SparkSession

path = kagglehub.dataset_download("sudalairajkumar/cryptocurrencypricehistory")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/cryptocurrencypricehistory


In [51]:
spark = SparkSession.builder.getOrCreate()

In [52]:
spark

In [53]:
for file in os.listdir(path):
    print(file)

coin_ChainLink.csv
coin_Cardano.csv
coin_Solana.csv
coin_Dogecoin.csv
coin_Polkadot.csv
coin_NEM.csv
coin_XRP.csv
coin_Ethereum.csv
coin_Aave.csv
coin_Bitcoin.csv
coin_Cosmos.csv
coin_Litecoin.csv
coin_Uniswap.csv
coin_EOS.csv
coin_BinanceCoin.csv
coin_CryptocomCoin.csv
coin_USDCoin.csv
coin_Monero.csv
coin_Tron.csv
coin_WrappedBitcoin.csv
coin_Tether.csv
coin_Iota.csv
coin_Stellar.csv


In [54]:
Coins = ["BTC", "ETH", "DOGE", "USD", "Stellar"]

Read_BTC = spark.read.csv(f"{path}/coin_Bitcoin.csv", header=True, inferSchema=True)
Read_ETH = spark.read.csv(f"{path}/coin_Ethereum.csv", header=True, inferSchema=True)
Read_DOGE = spark.read.csv(f"{path}/coin_Dogecoin.csv", header=True, inferSchema=True)
Read_USD = spark.read.csv(f"{path}/coin_USDCoin.csv", header=True, inferSchema=True)
Read_Stellar = spark.read.csv(f"{path}/coin_Stellar.csv", header=True, inferSchema=True)

print("Data has been successfully read")

Data has been successfully read


In [55]:
print("Bitcoin Data")
Read_BTC.show(5)

print("Ethereum Data")
Read_ETH.show(5)

print("Dogecoin Data")
Read_DOGE.show(5)

print("USDCoin Data")
Read_USD.show(5)

print("Stellar Data")
Read_Stellar.show(5)

Bitcoin Data
+---+-------+------+-------------------+------------------+------------------+------------------+------------------+------+---------------+
|SNo|   Name|Symbol|               Date|              High|               Low|              Open|             Close|Volume|      Marketcap|
+---+-------+------+-------------------+------------------+------------------+------------------+------------------+------+---------------+
|  1|Bitcoin|   BTC|2013-04-29 23:59:59|147.48800659179688|             134.0|134.44400024414062| 144.5399932861328|   0.0| 1.6037688645E9|
|  2|Bitcoin|   BTC|2013-04-30 23:59:59|146.92999267578125| 134.0500030517578|             144.0|             139.0|   0.0|  1.542813125E9|
|  3|Bitcoin|   BTC|2013-05-01 23:59:59|139.88999938964844|107.72000122070312|             139.0|116.98999786376953|   0.0|1.29895459375E9|
|  4|Bitcoin|   BTC|2013-05-02 23:59:59| 125.5999984741211| 92.28189849853516|116.37999725341797|105.20999908447266|   0.0|1.16851749525E9|
|  5|Bi

In [62]:
Read_BTC.createOrReplaceTempView("BTC")
Read_ETH.createOrReplaceTempView("ETH")
Read_DOGE.createOrReplaceTempView("DOGE")
Read_USD.createOrReplaceTempView("USD")
Read_Stellar.createOrReplaceTempView("Stellar")

In [70]:
sql_query = """
SELECT 'Bitcoin' AS Coin, AVG(Close) AS Harga_Penutupan_Bulanan FROM BTC
UNION ALL
SELECT 'Ethereum' AS Coin, AVG(Close) AS Harga_Penutupan_Bulanan FROM ETH
UNION ALL
SELECT 'Dogecoin' AS Coin, AVG(Close) AS Harga_Penutupan_Bulanan FROM DOGE
UNION ALL
SELECT 'USDCoin' AS Coin, AVG(Close) AS Harga_Penutupan_Bulanan FROM USD
UNION ALL
SELECT 'Stellar' AS Coin, AVG(Close) AS Harga_Penutupan_Bulanan FROM Stellar
"""

Average = spark.sql(sql_query)
Average.show(truncate=False)

+--------+-----------------------+
|Coin    |Harga_Penutupan_Bulanan|
+--------+-----------------------+
|Bitcoin |6711.290443071496      |
|Ethereum|383.9106913193636      |
|Dogecoin|0.013762639322080631   |
|USDCoin |1.0037913488368773     |
|Stellar |0.10150907766757886    |
+--------+-----------------------+

